In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, GroupKFold
import xgboost as xgb
from datetime import datetime, timedelta
import gc
import optuna
from utils_testing import optuna_logging
import pytz
UTC = pytz.utc  
timeZ_Kl = pytz.timezone('Asia/Kolkata')

In [ ]:
train_df = pd.read_pickle("../data/train_df_interim.pickle")
test_df = pd.read_pickle("../data/test_df_interim.pickle")

train_df.shape, test_df.shape

In [ ]:
drop = ['SURV_DTE'
        , 'sand_target_avg'
        , 'fold'
       ]
target = 'PCT_DESAT_TO_ORIG'
indep = train_df.columns.difference(drop+[target])
indep_master = indep.copy() # Taking a copy so it can be used to get the original features
indep

# Xgboost

### 5 fold Groupd CV

In [ ]:
def xgb_eval_rmse(preds, dtrain):
    actual = dtrain.get_label()
    preds = np.where(preds>=1,1, preds)
    preds = np.where(preds<=0,0, preds)
    
    fold_rmse = np.sqrt(mean_squared_error(actual, preds))
    
    return 'xgb_eval_rmse', fold_rmse

In [ ]:
def train_xgb_model(train_df, xgb_params):
    
    num_rounds = 100000

    fold_iterations = []
    fold_results = []
    xgb_models_fold = {}

    print("")
    for fold_i in range(0, train_df.fold.max()+1):

        train_fold = train_df[train_df.fold!=fold_i].copy()
        valid_fold = train_df[train_df.fold==fold_i].copy()

        dtrain_local = xgb.DMatrix(data= train_fold[indep] , label=train_fold[target])
        dtest_local = xgb.DMatrix(data= valid_fold[indep] , label=valid_fold[target])

        eval_set = [(dtrain_local,'train'), (dtest_local,'test')]

        np.random.seed(100)
        xgb_model_local = xgb.train(xgb_params,
                                    dtrain_local,
                                    evals = eval_set,
                                    num_boost_round = num_rounds,
#                                     feval = xgb_eval_rmse
#                                     maximize = False,
                                    verbose_eval = False,
                                    early_stopping_rounds = 50)
        xgb_local_prediction = xgb_model_local.predict(dtest_local)

        xgb_local_prediction = np.where(xgb_local_prediction<0, 0, xgb_local_prediction)
        xgb_local_prediction = np.where(xgb_local_prediction>1, 1, xgb_local_prediction)

        fold_rmse = np.sqrt(mean_squared_error(valid_fold[target], xgb_local_prediction))
        fold_iteration = xgb_model_local.best_iteration
        
        fold_iterations.append(fold_iteration)
        fold_results.append(np.round(fold_rmse, 5))
        xgb_models_fold[fold_i] = xgb_model_local
        
        print(f"Current fold: {fold_i}, iteration {fold_iteration}, RMSE {fold_rmse}")
    
    return fold_iterations, fold_results, xgb_models_fold

# XGB optuna

In [ ]:
def train_xgb_model_optuna(trial):
    """
    This function is used to train the model using the parameters obtained from optuna.
    """
    xgb_param = {
                'objective' : 'reg:squarederror'
                , 'eval_metric': 'rmse'
                , 'max_depth' : trial.suggest_int("max_depth", 3, 7)
                , 'eta': trial.suggest_float("eta", 0.01, 0.1, log=True)
                , 'colsample_bytree': trial.suggest_float("colsample_bytree", 0.8, 1.0)
                , 'subsample': trial.suggest_float("subsample", 0.8, 1.0)
                , 'min_child_weight': trial.suggest_float("min_child_weight", 0, 20)
            }
    

    xgb_fold_iterations, xgb_fold_results, xgb_models_fold = train_xgb_model(train_df=train_df, 
                                                                             xgb_params = xgb_param)
    
    avg_error = np.mean(xgb_fold_results)
    print("Avg.Fold results:", avg_error)

    return avg_error

In [ ]:
# Optuna Hyper-parameter tuning
xgb_study = optuna.create_study(direction="minimize")
xgb_study.optimize(train_xgb_model_optuna
                   , n_trials=50
                   , n_jobs=1
                   #                , timeout=600
                   , show_progress_bar=True
                   , gc_after_trial=True
              )

# Write the best hyer-parameter and the best RMSE to the logging file
optuna_logging(model='xgb', study=xgb_study, indep=np.array(indep))

print("Number of finished trials: ", len(xgb_study.trials))
print("Best trial:", xgb_study.best_trial.number)
print("Best Value: {}".format(xgb_study.best_trial.value))
print("Params: ")
xgb_study.best_params

In [ ]:
# Read all the hyperparameters and their best RMSE from the logged file
filename = f"../Optuna_logging/xgb_optuna_logging.csv"
temp = pd.read_csv(filename)
temp

In [ ]:
best_RMSE = temp.best_RMSE.min()
xgb_params = eval(temp.best_param[temp.best_RMSE==best_RMSE].values[0])
print(f"The parameter corresponding to the best RMSE {best_RMSE}")
xgb_params

# Indep combination

In [ ]:
def get_indep_combination(indep_all_combo, total_combinations_to_try):
    """
    This function trains the XGB model based on the different combinations of independent 
    features from the overall features that is available and write it to the file 
    xgb_best_indep_combo.csv
    """
    
    xgb_params = {'objective' : 'reg:squarederror'
                  ,'eval_metric': 'rmse'
                  ,'max_depth' : 5
                  ,'eta' : 0.01
                  ,'subsample': 0.9
                  ,'colsample_bytree': 0.9
                  ,'min_child_weight':20
                  ,'gamma': 1
        #           ,'tree_method' : 'gpu_hist'
                  }

    # reading the iterations ran so far
    global overall_best
    overall_best = pd.read_csv("../indep_combo/xgb_best_indep_combo.csv")
#     overall_best['indep'] = overall_best.indep.apply(lambda x : eval(x))

    random_index = np.random.choice(len(indep_all_combo), total_combinations_to_try, replace=False)
    mean_fold_result = []
    best_result={}
    indep_df = []

    # declare the indep as global so the changes can be reflected in the training function
    global indep 
    
    best = 10000
    for i, indep_ind in enumerate(random_index):
        indep= indep_all_combo[indep_ind]
        print(f"{i}/{total_combinations_to_try}")

        fold_iterations, fold_results, xgb_models_fold = train_xgb_model(train_df=train_df,
                                                                         xgb_params = xgb_params)
        mean_fold_result.append(np.mean(fold_results))
        indep_df.append(indep)
        avg_iteration = int(np.mean(fold_iterations))

        print("Fold iterations:", fold_iterations)
        print("Average iteration:", avg_iteration)
        print("Fold results:", fold_results)
        print("Avg.Fold results:", mean_fold_result[-1])

        # Printing the current best
        if mean_fold_result[-1]<best:
            best = mean_fold_result[-1]
            print(colored(f"New best {best}", 'green'))
            
            # Reading and writing the indep combo
            overall_best = pd.read_csv("../indep_combo/xgb_best_indep_combo.csv")
            best_indep = pd.DataFrame({'Date':datetime.now(timeZ_Kl).strftime('%d-%m-%Y %H:%M:%S'),
                                       'indep': str(indep), 
                                       'rmse': [best]})
            
            print(colored("writing the indep combos to disk", 'blue'))
            overall_best = overall_best.append(best_indep).drop_duplicates().reset_index(drop=True)
            overall_best.to_csv("../indep_combo/xgb_best_indep_combo.csv", index=False)
            
        else:
            print(colored(f"Best so far {best}", 'yellow'))

In [ ]:
print(f"Total actual features: {len(indep_master)}")

features_2_use=29
comb_features = combinations(indep_master, features_2_use)

indep_all_combo=[]
for indep_combo in list(comb_features):
    indep_all_combo.append(list(indep_combo))
    
print(f"Total features to use: {features_2_use}")
print(f"Total combo possible : {len(indep_all_combo)}")

In [ ]:
get_indep_combination(indep_all_combo=indep_all_combo, 
                      total_combinations_to_try=20)

In [ ]:
# Extract the parameters and the independent features with the best metric

days_before = 0

best_indep = pd.read_csv("../indep_combo/xgb_best_indep_combo.csv")
best_indep['Date'] = pd.to_datetime(best_indep.Date).dt.date.astype('str')

today_date = (datetime.now()-timedelta(days=days_before)).strftime('%Y-%m-%d')
print(today_date)

condition1 = (best_indep.Date==today_date)
best_indep = best_indep[condition1].reset_index(drop=True)

condition2 = (best_indep.rmse == best_indep.rmse.min())
indep = eval(best_indep[condition2].indep.values[0])
xgb_params = eval(best_indep[condition2].params.values[0])

print(f"Best RMSE : {best_indep.rmse.min()}")
print("Best indep size", len(indep))


### local

In [ ]:
xgb_params = {'objective' : 'reg:squarederror'
              ,'eval_metric': 'rmse'
              ,'max_depth' : 5
              ,'eta' : 0.01
              ,'subsample': 0.9
              ,'colsample_bytree': 0.9
              ,'min_child_weight':20
              ,'gamma': 1
    #           ,'tree_method' : 'gpu_hist'
              }

# xgb_params = {'objective' : 'reg:squarederror'
#               ,'eval_metric': 'rmse'
#               ,'max_depth' : 5
#               ,'eta' : 0.01
#               ,'subsample': 0.9
#               ,'colsample_bytree': 0.9
#               ,'min_child_weight':20
#               ,'gamma': 1
#     #           ,'tree_method' : 'gpu_hist'
#               }

# xgb_params['objective'] =  'reg:squarederror'
# xgb_params['eval_metric'] = 'rmse'

fold_iterations, fold_results, xgb_models_fold = train_xgb_model(train_df=train_df, 
                                                                 xgb_params = xgb_params)

avg_iteration = int(np.mean(fold_iterations))
print("Fold iterations:", fold_iterations)
print("Average iteration:", avg_iteration)
print("Fold results:", fold_results)
print("Avg.Fold results:", np.mean(fold_results))

In [ ]:
ind=4
xgb_imp = pd.DataFrame({'feature' : xgb_models_fold[ind].get_score().keys(), 
                        'fea_imp' : xgb_models_fold[ind].get_score().values()}).sort_values(['fea_imp'], ascending=False).reset_index(drop=True)
xgb_imp

In [ ]:
def fold_ensemble(model_list, test):
    """
    This is the Ensemble prediction of the final test data from the fold models
    """
    
    dtest_prod = xgb.DMatrix(data= test[indep])
    
    ens_pred = []
    for i in model_list.keys():
        print(f"Prediction for model {i}")  
        
        fold_pred = model_list[i].predict(dtest_prod)
        fold_pred = np.where(fold_pred<0, 0, fold_pred)
        fold_pred = np.where(fold_pred>1, 1, fold_pred)
        ens_pred.append(fold_pred)
        
    ensemble_prediction = np.array(ens_pred).mean(axis=0)
           
    return ensemble_prediction
        
# xgb_prod_prediction = fold_ensemble(model_list=xgb_models_fold, test=test_df)
# xgb_prod_prediction

### Prod

In [ ]:
dtrain_prod = xgb.DMatrix(data= train_df[indep] , label=train_df[target])
dtest_prod = xgb.DMatrix(data= test_df[indep])

train_prod_iter = avg_iteration #+ int(0.2*avg_iteration)
print(f"Training for {train_prod_iter} iterations")
np.random.seed(100)
xgb_model_prod = xgb.train(xgb_params,
                           dtrain_prod,
#                            evals = eval_set,
                           num_boost_round = train_prod_iter,
#                             feval = xgb_eval_rmspe,
#                             maximize = False,
#                            verbose_eval = True,
#                            early_stopping_rounds = 50
                          )

In [ ]:
xgb_prod_prediction = xgb_model_prod.predict(dtest_prod)
xgb_prod_prediction = np.where(xgb_prod_prediction<0, 0, xgb_prod_prediction)
xgb_prod_prediction = np.where(xgb_prod_prediction>1, 1, xgb_prod_prediction)
xgb_prod_prediction

In [ ]:
XGB_submission = pd.DataFrame({'PCT_DESAT_TO_ORIG':xgb_prod_prediction})
XGB_submission

In [ ]:
XGB_submission.to_csv("../sub/XGB_sub_38.csv", index=False)

# Model Explainability using SHAP values

In [ ]:
import shap
shap.initjs()

explainer = shap.TreeExplainer(xgb_model_prod)
shap_values = explainer.shap_values(train_df[indep].reset_index(drop=True))

In [ ]:
explainer = shap.TreeExplainer(xgb_model_prod)
shap_values = explainer.shap_values(train_df[indep].reset_index(drop=True))

i =10
shap.force_plot(explainer.expected_value, 
                shap_values[i], 
                features=train_df.loc[i, indep], 
                feature_names=train_df[indep].columns)

In [ ]:
shap.summary_plot(shap_values, 
                  features=train_df[indep].reset_index(drop=True),
                  feature_names=train_df[indep].columns)